In [1]:
import re
subitem_match = re.compile(r"^(?:([#]+)\s*)").match
magic_match = re.compile(r"^@lm\:\<(.+)\>").match
header_match = re.compile(r"^(?:([\*\#]*) )?([\+\*]+)([^\*\+]+?)\:?([\+\*]+)\:?\s*(.+)?$").match
frs_match = re.compile(r"([FU]RS\d+)\.?([\d\.]+)?").match
num_escape_match = re.compile(r"^(\*{2})(\#+)").match
image_caption_match = re.compile(r"(Image \d+)").match

class FRSParser():
    VALID_INDENTS = "*+"
    
    # inputs
    LINE_FRS = 0
    LINE_SUBITEM = 1
    LINE_OTHER = 2
    LINE_HEADER = 3
    LINE_NUM_ESCAPE = 4
    LINE_IMG_CAPTION = 5
    
    # states
    ST_INIT = 0
    ST_NONUM = 1
    ST_FRS = 2
    
    def __init__(self, depth=None, startval=1, indent_char="*"):
        self._startval = startval
        self.state = []
        self.ilvl = 0
        self.nlvl = 0 
        self.clvl = 0
        self.frs = ""
        self.nums = ""
        self.fmt = ""
        self.depth = depth
        if indent_char not in self.VALID_INDENTS:
            raise ValueError("Invalid Indent Character " + indent_char)
        self.indent_char = indent_char
        self.line = ""
        
        self.sm_st = self.ST_INIT
        self.sm = [
            # ST_INIT             ST_NONUM             ST_FRS
            [self.set_frs,        self.set_frs,        self.set_frs],        # LINE_FRS
            [None,                None,                self.set_subitem],    # LINE_SUBITEM
            [None,                None,                None],                # LINE_OTHER
            [self.set_header,     self.set_header,     self.set_header],     # LINE_HEADER
            [self.set_num_escape, self.set_num_escape, self.set_num_escape], # LINE_NUM_ESCAPE
            [None,                None,                None],                # LINE_IMG_CAPTION
        ]
        
    def _grow(self, n):
        while len(self.state) < n:
            self.state.append(self._startval)
        
    def indent(self, n=1):
        maxn = self.clvl + self.nlvl + n
        minn = self.clvl + self.nlvl
        self._grow(maxn)
        for i in range(minn, maxn):
            self.state[i] = 1
        self.clvl += n
    
    def dedent(self, n=1):
        self.clvl -= n
        self.next()
        
    def next(self):
        self.state[self.clvl+self.nlvl-1] += 1
        
    def set_indent(self, n):
        diff = n - (self.ilvl + self.clvl)
        if diff == 0:
            self.next()
        elif diff > 0:
            self.indent(diff)
        else:
            self.dedent(-diff)
            
    def curr_state(self, lvl):
        return self.state[:self.nlvl + lvl]
    
    def format_lvl(self, lvl):
        idl = lvl + self.ilvl
        if idl < 0: idl = 0
        ind = self.indent_char * idl
        if self.depth is not None and idl > self.depth:
            return ind
        space = " " if ind else ""
        frs = self.frs
        fmt = "*"
        nums = ".".join(str(n) for n in self.curr_state(lvl))
        dot = "." if nums else ""
        post = ":*"
        return "".join((ind, space, fmt, frs, dot, nums, post))
    
    def set_state_from_nums(self, nums):
        if not nums:
            lnums = []
        else:
            lnums = nums.split(".")
        self.state = [self._startval] * len(lnums)
        for i, n in enumerate(lnums):
            self.state[i] = int(n)
        return len(lnums)
    
    def feed(self, line):
        ev, arg = self._parse_input(line)
        func = self.sm[ev][self.sm_st]
        if func:
            st, rv = func(arg)
            self.sm_st = st
        else:
            rv = line
        return rv
    
    def _parse_input(self, line): 
        """Parse the input line to determine the 
        type of input, and any arguments to be passed
        to the input event handler.
        """
        m = header_match(line)
        if m:
            ind, fmt, s, fmt2, _ = m.groups()

            # Verify format to avoid false positive
            # if other * or + characters appear in the string
            if fmt == fmt2[::-1]:
                m2 = frs_match(s)
                if m2:
                    typ = self.LINE_FRS
                    frs, nums = m2.groups()
                    arg = (line, ind, fmt, frs, nums)
                    return typ, arg
                if image_caption_match(s):
                    return self.LINE_IMG_CAPTION, ()
                else:
                    typ = self.LINE_HEADER
                    arg = (line, ind, fmt, s)
                return typ, arg
        m = subitem_match(line)
        if m:
            typ = self.LINE_SUBITEM
            arg = line, len(m.group(1) or "")
            return typ, arg
        m = num_escape_match(line)
        if m:
            typ = self.LINE_NUM_ESCAPE
            arg = line,
            return typ, arg
        return self.LINE_OTHER, (line,)
    
    def frs_string(self):
        return self.format_lvl(self.clvl)
    
    def parse_text(self, text):
        for line in text.splitlines():
            yield self.feed(line)
    
    # State machine methods
                                
    def set_frs(self, arg):
        line, indent, fmt, frs, nums = arg
        self.ilvl = len(indent or "")
        self.fmt = fmt
        self.frs = frs
        self.nlvl = self.set_state_from_nums(nums)
        self.clvl = 0
        return self.ST_FRS, line
    
    def set_header(self, arg):
        return self.ST_NONUM, arg[0]
    
    def set_subitem(self, arg):
        line, n = arg
        self.set_indent(n)
        fmt = self.frs_string()
        try:
            _, tail = line.split(" ", 1)
        except ValueError:
            v = fmt
        else:
            v = " ".join((fmt, tail))
        return self.sm_st, v
    
    def set_num_escape(self, arg):
        line, = arg
        return self.sm_st, line.lstrip("*")
    
NumState = FRSParser

In [3]:
ntests = 0
passed = 0
errors = []
def assert_ab(a, b, equal=True):
    global ntests, passed, errors
    ntests += 1
    try:
        if equal:
            assert a == b, "%r != %r" % (a, b)
        else:
            assert a != b, "%r == %r" % (a, b)
    except AssertionError as e:
        val = str(e)
        if isinstance(a, str) and isinstance(b, str):
            val += "\n" + str(a.splitlines()) + "\n" + str(b.splitlines())
        errors.append(val)
    else:
        passed += 1

def assert_header_match(s, exp):
    assert_ab(header_match(s).groups(), exp)
    
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("*URS*", (None, "*", "URS", "*", None))
assert_header_match("*FRS1234*", (None, "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*", ("*", "*", "FRS1234", "*", None))
assert_header_match("*FRS1234.5.6*", (None, "*", "FRS1234.5.6", "*", None))
assert_header_match("*+FRS1234.5.6+*", (None, "*+", "FRS1234.5.6", "+*", None))
assert_header_match("** *+FRS1234.5.6.7.8+*", ("**", "*+", "FRS1234.5.6.7.8", "+*", None))
assert_header_match("* *FRS1234:*", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*:", ("*", "*", "FRS1234", "*", None))
assert_header_match("* *FRS1234*: Bob", ("*", "*", "FRS1234", "*", "Bob"))
assert header_match("**") == None
assert header_match("***") == None

def assert_num_escape(s, exp):
    m = num_escape_match(s)
    r = m.groups() if m else None
    assert_ab(r, exp)
    
assert_num_escape("**# Bob", ("**", "#"))
assert_num_escape("**## Bob", ("**", "##"))
assert_num_escape("**### Bob", ("**", "###"))
assert_num_escape("# Bob", None)
assert_num_escape("## Bob", None)
assert_num_escape("### Bob", None)
assert_num_escape("* Bob", None)
assert_num_escape("** Bob", None)
assert_num_escape("*** Bob", None)

#def assert_img_match(s):
#    m = image_match(s)
#    assert_ab(bool(m), True)
    
#assert_img_match("Image 1")

def set_str(st, s):
    ind, fmt, fs, _, _ = header_match(s).groups()
    frs, nums = frs_match(fs).groups()
    st.set_frs((s, ind, fmt, frs, nums))
    

def test_state(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_state(s, statevals, ilvl, nlvl):
    state = test_state(s)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.state, statevals)

def test_ilvl(s, ilvl):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_ilvl(s, ilvl):
    state = test_ilvl(s, ilvl)
    assert_ab(state.ilvl, ilvl)
    
def test_fmt(s):
    state = NumState()
    set_str(state, s)
    return state
    
def assert_fmt1(s, id, fmt):
    state = test_fmt(s)
    state.indent(id)
    assert_ab(state.frs_string(), fmt)
    
def assert_fmt2(s, id, fmt):
    state = test_fmt(s)
    for _ in range(id):
        state.indent(1)
    assert_ab(state.frs_string(), fmt)
    
def assert_indent(s, id, fmt):
    assert_fmt1(s, id, fmt)
    assert_fmt2(s, id, fmt)

assert_ilvl("*FRS1234*", 0)
assert_ilvl("*FRS1234.5.6*", 0)
assert_ilvl("** *FRS1234.5.6*", 2)

assert_state("*FRS1234*", [], 0, 0)
assert_state("*FRS1234.5.6*", [5, 6], 0, 2)
assert_state("** *FRS1234.5.6*", [5, 6], 2, 2)
assert_state("** *FRS1234*", [], 2, 0)

def assert_state2(s, ilvl, nlvl, clvl):
    state = test_state(s)
    st = state.state.copy()
    state.indent()
    st.append(1)
    assert_ab(state.state, st)
    assert_ab(state.ilvl, ilvl)
    assert_ab(state.nlvl, nlvl)
    assert_ab(state.clvl, clvl)

assert_state2("*FRS1234*", 0, 0, 1)
assert_state2("* *FRS1234*", 1, 0, 1)
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("*FRS1234*", 1, "* *FRS1234.1:*")
assert_indent("* *FRS1234*", 1, "** *FRS1234.1:*")
assert_indent("*FRS1234.1*", 1, "* *FRS1234.1.1:*")
assert_indent("*FRS1234.1*", 2, "** *FRS1234.1.1.1:*")

def assert_next(s, exp):
    st = NumState()
    set_str(st, s)
    st.next()
    assert_ab(st.frs_string(), exp)

def assert_dedent(s, id, nxt, dd, fmt, typ=1):
    st = NumState()
    set_str(st, s)
    st.indent(id)
    for _ in range(nxt):
        st.next()
    if typ == 1:
        st.dedent(dd)
    else:
        for _ in range(dd):
            st.dedent(1)
    assert_ab(st.frs_string(), fmt)

assert_next("*FRS1234.1*", "*FRS1234.2:*")
assert_next("* *FRS1234.1*", "* *FRS1234.2:*")
assert_next("*FRS1234.1.2*", "*FRS1234.1.3:*")
assert_next("* *FRS1234.1.2*", "* *FRS1234.1.3:*")
assert_dedent("*FRS1234.1*", 1, 2, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 1, 1, 1, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 1, "* *FRS1234.1.2:*")
assert_dedent("*FRS1234.1*", 2, 1, 2, "*FRS1234.2:*")
assert_dedent("*FRS1234.1*", 3, 1, 2, "* *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 1, 2, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 1, 1, 1, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 1, "** *FRS1234.1.2:*")
assert_dedent("* *FRS1234.1*", 2, 1, 2, "* *FRS1234.2:*")
assert_dedent("* *FRS1234.1*", 3, 1, 2, "** *FRS1234.1.2:*")

def assert_feed_indent(s, feed, exp):
    st = NumState()
    set_str(st, s)
    st.sm_st = st.ST_FRS
    st.feed(feed)
    assert_ab(st.frs_string(), exp)
    
assert_feed_indent("*FRS1234*", "# Bob", "* *FRS1234.1:*")
assert_feed_indent("*FRS1234*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "# Bob", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## Bob", "** *FRS1234.1.1.1:*")
assert_feed_indent("* *FRS1234.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("* *FRS1234*", "## Bob", "** *FRS1234.1:*")
assert_feed_indent("** *FRS1234.1*", "## Bob", "** *FRS1234.2:*")
assert_feed_indent("* *FRS1234.1*", "## Bob", "** *FRS1234.1.1:*")
assert_feed_indent("** *FRS1234.1.1*", "# Bob", "* *FRS1234.2:*")
assert_feed_indent("*FRS1234.1*", "# ", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "## ", "** *FRS1234.1.1.1:*")
assert_feed_indent("*FRS1234.1*", "#", "* *FRS1234.1.1:*")
assert_feed_indent("*FRS1234.1*", "##", "** *FRS1234.1.1.1:*")

def assert_double_parse(t, d=5):
    r = list(NumState(d).parse_text(t))
    r2 = list(NumState(d).parse_text("\n".join(r)))
    assert_ab(r, r2)
    
dpt = """*FRS2881.4*
# PBSUsers.conf file will keep track of the following data:
## Per User
### User Name
### Group
### Password
### Email Address
### Date Password was Saved
### Number of Failed Login Attempts
## Per Group
### Group Name
### Password Expiration Period (Days)
## 2-D Permission Table
### Each row corresponds to each group"""
assert_double_parse(dpt)

def assert_new_state(s, exp, nxt=1, ind=0):
    st = NumState()
    for l in s.splitlines():
        st.feed(l)
    for _ in range(nxt):
        st.next()
    for _ in range(ind):
        st.indent()
    assert_ab(st.frs_string(), exp)
    
assert_new_state("*FRS123.1*\n*FRS31.2*", "*FRS31.3:*")
assert_new_state("*FRS123.1*\n*FRS123.3*", "*FRS123.4:*")

def assert_parse_text(txt, exp):
    st = NumState()
    lines = list(st.parse_text(txt))
    res = "\n".join(lines)
    assert_ab(res, exp)

lines = """*FRS1234.1:*
#"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*"""
assert_parse_text(lines, exp)

lines="""*FRS1234.1:*
#
## Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
#
##
"""
exp = """*FRS1234.1:*
* *FRS1234.1.1:*
** *FRS1234.1.1.1:* Item two!
*** Non-numbered FRS text or sub-item
*FRS1234.2:*
Foobar
!Foobar.jpg!
* *FRS1234.2.1:*
** *FRS1234.2.1.1:*"""

assert_parse_text(lines, exp)

lines = "+*FRS*+\n*FRS1234:*\n#"
exp = "+*FRS*+\n*FRS1234:*\n* *FRS1234.1:*"
assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
*** Bob
## Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
*** Bob
** *FRS2908.1.1.1:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.1*
# Fred
#### Biff
*** Bob
# Zoe"""

exp = """*FRS2908.1*
* *FRS2908.1.1:* Fred
**** *FRS2908.1.1.1.1.1:* Biff
*** Bob
* *FRS2908.1.2:* Zoe"""

assert_parse_text(lines, exp)

lines = """*FRS2908.0*
# Bob1
## Bob2
*** Fred1
*** Fred2
## Bob3
## Bob4
## Bob5
*** Fred3
#### Bob6
#### Bob7
#### Bob8
*** Fred4
# Bob9
"""

exp = """*FRS2908.0*
* *FRS2908.0.1:* Bob1
** *FRS2908.0.1.1:* Bob2
*** Fred1
*** Fred2
** *FRS2908.0.1.2:* Bob3
** *FRS2908.0.1.3:* Bob4
** *FRS2908.0.1.4:* Bob5
*** Fred3
**** *FRS2908.0.1.4.1.1:* Bob6
**** *FRS2908.0.1.4.1.2:* Bob7
**** *FRS2908.0.1.4.1.3:* Bob8
*** Fred4
* *FRS2908.0.2:* Bob9"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** Fred:
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** Fred:
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*FRS2908.0* 
* *FRS2908.0.1:* Bob
** considered +/- 15 degrees
* *FRS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*FRS123* 
# Bob
*Image 1*: Bob's image
# Bob"""

exp = """*FRS123* 
* *FRS123.1:* Bob
*Image 1*: Bob's image
* *FRS123.2:* Bob"""
assert_parse_text(lines, exp)

lines = """*URS2908.0* 
# Bob
** considered +/- 15 degrees
# Bob"""

exp = """*URS2908.0* 
* *URS2908.0.1:* Bob
** considered +/- 15 degrees
* *URS2908.0.2:* Bob"""
assert_parse_text(lines, exp)

def assert_parse_empty(s, exp):
    st = NumState()
    res = None
    for res in st.parse_text(s):
        pass
    assert_ab(res, exp)
    
assert_parse_empty("*User Requirements*\n# Bar", "# Bar")

def finish():
    print("%d/%d tests passed successfully" % (passed, ntests))
    for e in errors:
        print(e)
    if errors:
        raise Exception("Unit testing suite failed")
finish()

95/95 tests passed successfully


In [5]:
import clipboard

In [6]:
lines=r"""
*+FRS1754+*
# Each MFC shall use a calibration factor m and b used to convert raw voltage to actual flowrate
# Calibration of all MFCs shall be accessible through the Shell UI through manual input of calculated m and b values
# Old system variables related to LPM/V conversion shall be removed.
# No input controls or buttons related to calibration shall be present on the MFC Calibration screen
** User is required to perform external calibration and input the values.

*+Implementation+*
# Introduce m (LPM/V) and b (LPM) to the Cal Factors.cfg and 'cal' tab of the RIO Globals, for each MFC:
## CalMFCAir
### m (LPM/V)
### b (LPM)
## CalMFCCO2
### m (LPM/V)
### b (LPM)
## CalMFCO2
### m (LPM/V)
### b (LPM)
## CalMFCN2
### m (LPM/V)
### b (LPM)
# Wherever Voltage is currently converted to LPM by multiplying the Voltage by the MFC's LPM/V System Variable, change it so it multiplies the Voltage by slope, and then adds the intecept
# Wherever requested LPM is currently converted to Voltage by dividing the LPM by the MFC's LPM/V System Variable, change it so it subtracts intercept from LPM, and then divides by slope
# Remove the LPM/V System Variables, and the corresponding Globals from the Gases tab
# The New Calibration Editor is shown below:
!Cal_Sensor_MFC.png!
Numbered Mockup:
!Cal_Sensor_MFC_Numbered.png!
# Only 'Save' and 'Revert' buttons should be included for the MFC calibration menu.  Do not include the following buttons: 'tar1', 'tar2', '1-point', '2-point', 'zero', 'span'.
# Item 1 will be an additional radio button labelled Air, CO2, N2, and O2. When the user clicks on MFC, Item 1 will be shown in place of the 1-point, 2-point cal radio buttons.
## There will always be one item selected.
## When the program is first opened and navigated to MFC, Air will be selected.
"""
res = "\n".join(NumState(3).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+FRS1754+*
* *FRS1754.1:* Each MFC shall use a calibration factor m and b used to convert raw voltage to actual flowrate
* *FRS1754.2:* Calibration of all MFCs shall be accessible through the Shell UI through manual input of calculated m and b values
* *FRS1754.3:* Old system variables related to LPM/V conversion shall be removed.
* *FRS1754.4:* No input controls or buttons related to calibration shall be present on the MFC Calibration screen
** User is required to perform external calibration and input the values.

*+Implementation+*
# Introduce m (LPM/V) and b (LPM) to the Cal Factors.cfg and 'cal' tab of the RIO Globals, for each MFC:
## CalMFCAir
### m (LPM/V)
### b (LPM)
## CalMFCCO2
### m (LPM/V)
### b (LPM)
## CalMFCO2
### m (LPM/V)
### b (LPM)
## CalMFCN2
### m (LPM/V)
### b (LPM)
# Wherever Voltage is currently converted to LPM by multiplying the Voltage by the MFC's LPM/V System Variable, change it so it multiplies the Voltage by slope, and then adds the intecept
# Wherever

In [14]:
is_frs_item = re.compile(r"(?:[\*\#]* )?\*FRS([\d\.]+)\:?\*\:?(.*)").match
lines = clipboard.paste()
lines2 = []
for line in lines.splitlines():
    m = is_frs_item(line)
    if m:
        nums, text = m.groups()
        #line = nums.count(".") * "*" + (" *FRS%s:*" % nums) + text
        #print(line)
        line = "#" * nums.count(".") + text
    lines2.append(line)
#print("\n".join(lines2))
clipboard.copy("\n".join(lines2))

In [15]:
template=  r'''lines=r"""%s
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)
'''
s = template % clipboard.paste()
clipboard.copy(s)

In [17]:
lines=r"""*+User Requirements+*
See URS3194.2.6.5 in issue #3194

*+FRS2851+*
* *FRS2851.0*: Embedded metadata requirements:
## Reports generated by batch will include start/and stop information.
*** The first row will have the following text: Batch Name, Created By, Start Time, End Time
*** The second row will have the following information: Batch_Name, Created_By, Start_Time, Start_Time of the next batch from the Batch Table.
## Reports generated by time will include start time and stop time.
*** If a user elects to generate a report with a time span starting earlier than the selected database, and/or ending later than the selected database, there is currently no business need to correct them and attempt to coerce the start time and end time.
## Time stamps for edge cases during generation by batch:
### For an active DB with no 'next batch', use the current time as the 'End Time'
### For an archived DB with no 'next batch', use the start time of the next DB as the 'End Time'
# The getBatches call should return the list of batches in the active database, sorted in reverse chronological order.
# Reports generated from either the Web UI and the LabVIEW UI, the files should be located in the C:\Reports folder
# Reports layout
!Report_Generation.png!
*Image 1*: Reports Menu Mockup

!Report_Generation_Numbered.png!
*Image 2*: Numbered Reports Menu Mockup

# Appearance
## Item 1 - a listbox labelled Databases
### The list itself is a control, where 1 item is selectable
### The items in the list are to be indicators, not controls
### The list is to have 2 columns:
#### The first column is to be labeled "Date/Time Start"
#### The second column is to be labeled "Date/Time Stop"
## Item 2 - a listbox labeled Batches
### The list itself is a control, where 1 item is selectable
### The items in the list are to be indicators, not controls
### The list is to have 3 columns:
#### The first column is to be labeled "Batch Name"
#### The second column is to be labeled "Date/Time Start"
#### The third column is to be labeled "Date/Time Stop"
## Item 3 - a radio control button labeled Report Type
## Item 4 - a radio control button labeled Filter By
## Item 5&6 - Start Time and End Time selectors
### These will be timestamp controls
### The first will be labeled "Start Time"
### The second will be labeled "End Time"
### Both will be in the following format
#### First line: hh:mm:ss PM (instead of 24-hours, show AM or PM)
#### Second line: MM/DD/YYYY
### They will have a "Time/date browse button" tied to each, as displayed in the mockup
## Item 7 - a Boolean button labeled Generate
## Item 8 - a Boolean button labeled Go to File
## Item 9 - a string indicator labeled Status
## Item 10 - a string indicator labeled File 
# Item 1 - Databases list Functionality
## This list is to have 1 entry for each database in the Database folder.  Note that if a database is deleted from that folder, it should no longer appear in the list.
## The first column is to show the date and timestamp of when the database was started
## The second column is to show the date and timestamp of when the database was ended.  For the active database, since it still has not ended, show "--" in the second column.
## The list is to be sorted in reverse chronological order, with the active database on top, and the oldest databases on the bottom.
## This list is to be refreshed under the following conditions:
### When the layout is navigated to
### When the layout is active, and a database is archived and a new one is made active.  When this happens, the selected database must shift down by 1, so the same database is selected. 
## Selecting items
### Items in the list can be selected by clicking the item
### Only 1 item may be selected at a time
### 1 item must be selected at all times
### When an item is selected, it will be highlighted blue and its text will be white
### When the layout is first navigated to, the top item (i.e. the active database) will be selected.
### When a selection is made, it will trigger a refresh of item 2 (see below)
## If the list is longer than the display, users will be able to scroll up or down to see all the items.
## Users cannot change the text of the items in the list.
# Item 2 - Batches list Functionality
## This list is to have 1 entry for each batch in the selected database in the Databases list (item 2)
## The first column is to show the name of the batch
## The second column is to show the date and timestamp of when the batch was started
## The third column is to show the date and timestamp of when the batch was ended.
### For the active database's current batch, since it has not ended, show "--" in the second column.
### For an archived database, if the last batch was not actually ended in that database, show the end timestamp of the database itself
## The list is to be sorted in reverse chronological order, with the newest batch on top, and the oldest batch on the bottom.
## The list is to be refreshed under the following conditions:
### When the user first navigates to the layout, it must show the list of batches in the active database
### When the user selects a different database in the Databases list
### When the Databases list is refreshed (see item 2 above)
## Selecting items
### Items in the list can be selected by clicking the item
### Only 1 item may be selected at a time
### 1 item must be selected at all times
### When an item is selected, and the list is not grayed out, it will be highlighted blue and its text will be white
### When an item is selected, and the list is grayed out, it will be highlighted gray and its text will be white
### When the layout is first navigated to, the top item (i.e. the newest batch in the selected database) will be selected.
### When a different database is selected in item 2, the top item (i.e. the newest batch in the selected database) will be selected.
## When this item is grayed out, users will not be able to modify it.
# Item 3 - Report Type selector Functionality
## The options for this selector are Process Data, User Events, Errors, Alarms, and Recipe Steps
## When the layout is navigated to, Process Data will be selected
## Only 1 item can be selected at any time
## 1 item must be selected at all times
# Item 4 - Filter By selector Functionality
## The options for this selector are Batch, and Time
## When the layout is navigated to, Batch will be selected
## Only 1 item can be selected at any time
## 1 item must be selected at all times
## When the 'Batch' item is selected, item 2 is not grayed out, and item 5&6 are grayed out. 
## When the 'Time' item is selected, item 2 is grayed out, and item 6 is not.
# Item 5&6 - Start Time and End Time selectors functionality
## When this item is grayed out, users will not be able to modify it.
## When the layout is navigated to, the Start and End Time will be blank. 
# Item 7 - Generate button Functionality
## When this button is clicked, a report is generated with the selected parameters.
## While the report is being generated, the 'Status' indicator (item 9) will be updated
## After the report has been generated, its file name will be displayed in item 10
# Item 8 - Go to file button functionality
## This will only be visible and clickable if the File Name indicator (item 10) is populated with a report name (a string of digits with .csv type).  Otherwise, it is invisible. 
## When clicked, it will open the File Management VI, and highlight the latest file in the Reports folder
# Item 9 - Status indicator
## It will only be visible while a report is being generated.  Otherwise it will be hidden.
# Item 10 - File Name indicator functionality 
## When the layout is navigated to, this will not be visible.
## When a report is successfully generated in the layout, this will be visible.
## When visible, it will show the file name for the latest report successfully generated from the layout.
# If an error occurs during report generation, display the error dialog.
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+User Requirements+*
See URS3194.2.6.5 in issue #3194

*+FRS2851+*
* *FRS2851.0*: Embedded metadata requirements:
** *FRS2851.0.1:* Reports generated by batch will include start/and stop information.
*** The first row will have the following text: Batch Name, Created By, Start Time, End Time
*** The second row will have the following information: Batch_Name, Created_By, Start_Time, Start_Time of the next batch from the Batch Table.
** *FRS2851.0.2:* Reports generated by time will include start time and stop time.
*** If a user elects to generate a report with a time span starting earlier than the selected database, and/or ending later than the selected database, there is currently no business need to correct them and attempt to coerce the start time and end time.
** *FRS2851.0.3:* Time stamps for edge cases during generation by batch:
*** *FRS2851.0.3.1:* For an active DB with no 'next batch', use the current time as the 'End Time'
*** *FRS2851.0.3.2:* For an archived DB with no 'next

In [191]:
lines=r"""*+Current Behavior:+*
# In the following scenario, user1 will hijack user2's permission and "account" in the sense that whatever user1 does will be logged as user2's actions in the database.
## User1 logs in.
## User2 logs in.
## User2 logs out.
## User1 now inherits user2's name and Config setting privileges. User1's action is logged as user2 in the database.
# Login, logout, and user actions record the wrong usernames.

*+FRS2367+*
# All actions taken by users are checked for the correct user's permissions, then recorded as the correct user.  Even if more than 1 user is logged in at once.  Even if more than 1 user logs in at once, and then 1 or more log out.
# Users cannot hijack other users' sessions
# When a user logs in, all actions done by the user is attributed to that user and not another user or unknown user.

*+Implementation+*
Possible Solution:
Remove globals for sessions, session attributes, and local session with a shift register variable in webservice command handler.vi The local session will include items in the existing local session with session cookies ID and all the permissions for that user.

Webservice Command Handler should be modified to be more modular.

*+Notes:+*
Session cookie ID is randomly generated as far as I can tell. They do not increase serially.
The session currently does not end or get deleted because our code only adds session and does not delete them. However, once it exceeds 50 sessions, it will replace the oldest session with the newest one.
"""
res = "\n".join(NumState(7).parse_text(lines))
print(res)
import clipboard
clipboard.copy(res)


*+Current Behavior:+*
# In the following scenario, user1 will hijack user2's permission and "account" in the sense that whatever user1 does will be logged as user2's actions in the database.
## User1 logs in.
## User2 logs in.
## User2 logs out.
## User1 now inherits user2's name and Config setting privileges. User1's action is logged as user2 in the database.
# Login, logout, and user actions record the wrong usernames.

*+FRS2367+*
* *FRS2367.1:* All actions taken by users are checked for the correct user's permissions, then recorded as the correct user.  Even if more than 1 user is logged in at once.  Even if more than 1 user logs in at once, and then 1 or more log out.
* *FRS2367.2:* Users cannot hijack other users' sessions
* *FRS2367.3:* When a user logs in, all actions done by the user is attributed to that user and not another user or unknown user.

*+Implementation+*
Possible Solution:
Remove globals for sessions, session attributes, and local session with a shift register var